In [1]:
import torch
import numpy as np
from model_parameters import ModelParameters
from tqdm import tqdm
import time
from helper.torch_training import run_torch_sequence
from helper.scores import Metrics

# Data Loading

In [2]:
model_parameters = ModelParameters()
x_train, x_test, y_train, y_test = model_parameters.get_htru_2()

# Model definition

In [3]:
class ClassicCNN(torch.nn.Module):
    def __init__(self, num_features):
        super(ClassicCNN, self).__init__()
        self.conv1d = torch.nn.Conv1d(in_channels=num_features, out_channels=128, kernel_size=1)
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(128, 64)
        self.fc2 = torch.nn.Linear(64, 1)

    def forward(self, x):
        x = torch.reshape(x, (-1, 8,1))
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        x = torch.sigmoid(x)
        x = x.squeeze(1)
        return x

## Training CNN

In [4]:
metrics = Metrics()

model = ClassicCNN(model_parameters.num_features)
run_torch_sequence(model_parameters, model, x_train, y_train, x_test, y_test, metrics)
metrics.display()

100%|██████████| 6/6 [02:42<00:00, 27.06s/it]

Pour 6 runs:
Accuracy: 0.965 ± 0.017
Balanced Accuracy: 0.944 ± 0.033
Recall: 0.919 ± 0.067
Specificity: 0.969 ± 0.018
Precision: 0.770 ± 0.103
NPV: 0.992 ± 0.006
G-Mean: 0.943 ± 0.034
Informedness: 0.888 ± 0.065
Training Duration: 26.946 ± 2.018
Testing Duration: 0.000 ± 0.000
